# Let's fit a model on the housing price data!
1. Import packages
2. Load Data
3. Fit a LR model


# Import Packages

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load Data

In [ ]:
raw_data_train = pd.read_csv('https://raw.githubusercontent.com/jmpark0808/pl_mnist_example/main/train_hp_msci436.csv')
raw_data_train.tail()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125
1459,1460,20,RL,75.0,9937,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2008,WD,Normal,147500


In [ ]:
#Categorical Columns without 'ordered' categories
categorical_cols = [
   'MSZoning',
   'Street',
   'Alley',
   'LandContour',
   'LotConfig',
   'Neighborhood',
   'Condition1',
   'Condition2',
   'RoofStyle',
   'RoofMatl',
   'MasVnrType',
   'Foundation',
   'Heating',
   'GarageType',
   'SaleCondition',
]
#Mapping columns of 'ordered' categories to numbers
ordinal_cols = {
    'LotShape':{'Reg':1,'IR1':2,'IR2':2,'IR3':3},
    'Utilities':{'AllPub':1,'NoSewr':2,'NoSeWa':3,'ELO':4},
    'LandSlope':{'Gtl':1,'Mod':2,'Sev':3},
    'BldgType':{'1Fam':1,'2FmCon':2,'Duplx':3,'TwnhsE':4,'TwnhsI':5},
    'HouseStyle':{'1Story': 1,'1.5Fin': 2,'1.5Unf': 3,'2Story': 4,'2.5Fin': 5,'2.5Unf': 6,'SFoyer': 7,'SLvl': 8},
    'ExterQual':{'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1},
    'ExterCond':{'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1},
    'BsmtQual':{'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'NA': 0},
    'BsmtCond':{'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'NA': 0},
    'BsmtExposure':{'Gd': 4,'Av': 3,'Mn': 2,'No': 1,'NA': 0},
    'BsmtFinType1':{'GLQ': 6,'ALQ': 5,'BLQ': 4,'Rec': 3,'LwQ': 2,'Unf': 1,'NA': 0},
    'BsmtFinType2':{'GLQ': 6,'ALQ': 5,'BLQ': 4,'Rec': 3,'LwQ': 2,'Unf': 1,'NA': 0},
    'HeatingQC':{'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1},
    'CentralAir':{'N':0,'Y':1},
    'Electrical':{'SBrkr': 5,'FuseA': 4,'FuseF': 3,'FuseP': 2,'Mix': 1},
    'KitchenQual':{'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1},
    'Functional':{'Typ': 8,'Min1': 7,'Min2': 6,'Mod': 5,'Maj1': 4,'Maj2': 3,'Sev': 2,'Sal': 1},
    'FireplaceQu':{'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'NA': 0},
    'GarageFinish':{'Fin': 3,'RFn': 2,'Unf': 1,'NA': 0},
    'GarageQual':{'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'NA': 0},
    'GarageCond':{'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'NA': 0},
    'PavedDrive':{'Y':3,'P':2,'N':1},
    'PoolQC':{'Ex': 4,'Gd': 3,'TA': 2,'Fa': 1,'NA': 0},

}
#Categorical Variables that will not be converted nor used
cols_to_remove = [
    "MSSubClass", #this column is more accurately described by 'BldgType' and ''HouseStyle'
    'Exterior1st', #this column is not ordinal and has far too many categories to map to binary columns
    'Exterior2nd', #this column is not ordinal and has far too many categories to map to binary columns
    'MiscFeature', #this column is more accurately described by 'MiscVal': $Value of miscellaneous feature
    'Fence', #this column has non-exclusive categories and can therefore not accurately describe a fence
    'SaleType', #this column has non-exclusive categories and can therefore not accurately describe a home sale
]

# **Data Cleaning & Encoding**

In [ ]:
#Function to create binary columns for each categorical but non-ordinal column
def convert_categorical_to_binary(df, columns):
    for column in columns:
        if column in df.columns:
            df_encoded = pd.get_dummies(df[column], prefix=column, dtype='int64')
            df = pd.concat([df, df_encoded], axis=1)
            df = df.drop(column, axis=1)
    return df
#Function to map ordinal categories to numerical values
def replace_categorical_with_numerical(df, column, mapping):
    if column in df.columns:
        df[column] = df[column].replace(mapping)
    return df

In [ ]:
#Convert Categorical cols into a binary col for each category
new_train = convert_categorical_to_binary(raw_data_train, categorical_cols)
#Loop over ordinal categories and convert them to numerical
for col in ordinal_cols:
    new_train = replace_categorical_with_numerical(new_train, col, ordinal_cols[col])

In [ ]:
new_train.tail()

,Id,MSSubClass,LotFrontage,LotArea,LotShape,Utilities,LandSlope,BldgType,HouseStyle,OverallQual,...,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1455,1456,60,62.0,7917,1,1,1,1,4,6,...,0,0,0,0,0,0,0,0,1,0
1456,1457,20,85.0,13175,1,1,1,1,1,6,...,0,0,0,0,0,0,0,0,1,0
1457,1458,70,66.0,9042,1,1,1,1,4,7,...,0,0,0,0,0,0,0,0,1,0
1458,1459,20,68.0,9717,1,1,1,1,1,5,...,0,0,0,0,0,0,0,0,1,0
1459,1460,20,75.0,9937,1,1,1,1,1,5,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
print(raw_data_train.shape)
print(new_train.shape)

(1460, 81)
(1460, 168)


# **Select Columns**

In [ ]:
df = new_train.select_dtypes(include = ['float64', 'int64']).fillna(0)
y = df['SalePrice'].values
X = df.drop('SalePrice',axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
print(X)

[[1.000e+00 6.000e+01 6.500e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [2.000e+00 2.000e+01 8.000e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [3.000e+00 6.000e+01 6.800e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 ...
 [1.458e+03 7.000e+01 6.600e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [1.459e+03 2.000e+01 6.800e+01 ... 0.000e+00 1.000e+00 0.000e+00]
 [1.460e+03 2.000e+01 7.500e+01 ... 0.000e+00 1.000e+00 0.000e+00]]


In [ ]:
print(df.columns)

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'LotShape', 'Utilities',
       'LandSlope', 'HouseStyle', 'OverallQual', 'OverallCond',
       ...
       'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort',
       'GarageType_Detchd', 'SaleCondition_Abnorml', 'SaleCondition_AdjLand',
       'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal',
       'SaleCondition_Partial'],
      dtype='object', length=162)


# Fit a Linear Regression Model

In [ ]:
reg = LinearRegression().fit(X_train, y_train)
a = reg.coef_
reg.coef_



array([-7.39289196e-02, -1.72255173e+02,  4.14820140e+01,  5.93634791e-01,
       -1.27228189e+03, -3.40228085e+04,  4.94324675e+02,  1.66201063e+02,
        6.82412191e+03,  5.47796751e+03,  1.85904387e+02, -5.17678931e+01,
        2.81771958e+01,  9.15571130e+03, -3.87532079e+03,  3.92593222e+03,
       -4.41279369e+03,  6.07757410e+03,  3.17003426e+02,  1.76807674e+01,
        2.35777282e+03, -1.59070550e+00, -2.39601843e+00,  1.36940381e+01,
        2.53656653e+03,  2.45644103e+03, -3.37685142e+03,  1.84121414e+01,
        2.08239536e+01, -1.14917418e+01,  2.77443750e+01, -1.10739331e+03,
        5.68407479e+03,  1.24618789e+03,  3.57114625e+03, -3.24036362e+03,
       -9.71536149e+03,  5.84776622e+03,  3.41016760e+03,  4.86881529e+03,
        4.40219377e+03, -6.67660741e+02, -1.06163224e+02,  1.01766790e+03,
        2.81104180e+03,  3.77278328e+01,  5.76322573e+03, -3.92049374e+03,
       -7.97895749e+02,  1.07828182e+01,  2.00500367e+01,  7.25762965e+00,
        2.75779705e+01,  

# **Calculate score with all features**

In [ ]:
#Train set Score
reg.score(X_test,y_test)

0.8669434433555137

In [ ]:
#Test set Score
reg.score(X_train,y_train)

0.9136921727014607

# **Sort columns by their net effect (mean value multiplied by coefficient)**

In [ ]:
import statistics

In [ ]:
#Since this is far too many variable to ask the user to input, we will determine the top 10 inputs
i = 0
df_no_price = df.drop('SalePrice',axis=1)
df_no_id_or_price = df_no_price.drop('Id',axis=1)
col_weights = pd.DataFrame(columns=['Col','Effect'])
for col in df_no_id_or_price:
    col_weights.loc[len(col_weights)] = [col,abs(statistics.mean(df[col])*a[i])]
    i += 1
col_weights = col_weights.sort_values(by=['Effect'], ascending=False)
col_weights.head(n=20)

,Col,Effect
9,YearBuilt,1.079854e+07
26,1stFlrSF,3.926018e+06
55,MiscVal,1.847574e+06
57,YrSold,1.616730e+06
44,GarageArea,1.329567e+06
41,GarageYrBlt,1.247684e+06
125,RoofMatl_CompShg,4.904428e+05
2,LotArea,4.362592e+05
10,YearRemodAdd,3.689953e+05
18,BsmtFinSF1,1.406353e+05


# **Visualize Importance of Features**

In [ ]:
print(col_weights['Col'].tolist())

['YearBuilt', '1stFlrSF', 'MiscVal', 'YrSold', 'GarageArea', 'GarageYrBlt', 'RoofMatl_CompShg', 'LotArea', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'GarageType_Attchd', 'WoodDeckSF', 'GarageType_Detchd', 'TotRmsAbvGrd', 'OverallCond', 'LandSlope', 'KitchenQual', 'ExterCond', 'Functional', 'BsmtFinType1', 'GrLivArea', 'GarageCond', 'Heating_GasA', 'BsmtQual', 'SaleCondition_Abnorml', 'LotConfig_Inside', 'Electrical', 'BsmtCond', 'GarageType_BuiltIn', 'PavedDrive', 'LandContour_Lvl', 'BedroomAbvGr', 'LotFrontage', 'Condition2_Norm', 'FullBath', 'FireplaceQu', 'MSZoning_RL', 'SaleCondition_Normal', 'BsmtExposure', '2ndFlrSF', 'MasVnrType_None', 'MasVnrType_BrkFace', 'MasVnrArea', 'RoofStyle_Gable', 'Condition1_Norm', 'KitchenAbvGr', 'Fireplaces', 'Street_Pave', 'Neighborhood_OldTown', 'TotalBsmtSF', 'CentralAir', 'GarageType_Basment', 'MasVnrType_BrkCmn', 'GarageCars', 'Neighborhood_NAmes', 'Foundation_CBlock', 'Foundation_PConc', 'Neighborhood_Timber', 'Utilities', 'HouseStyle', 'Neig

In [ ]:
inclusion_cols_ordered = [
    'YearBuilt',
    '1stFlrSF',
    'GarageYrBlt',
    'GarageArea',
    'MiscVal',
    'YrSold',
    'RoofMatl',
    'YearRemodAdd',
    'LotArea',
    'KitchenQual',
    'TotRmsAbvGrd',
    'OverallCond',
    'GrLivArea',
    'LandSlope',
    'Functional',
    'Condition2',
    'BsmtFinType1',
    'ExterCond',
    'GarageCond',
    'BsmtQual',
    'LotConfig',
    'BsmtCond',
    'FireplaceQu',
    'Street',
    'BsmtFinSF1',
    'LotFrontage',
    'BsmtExposure',
    'GarageType',
    '2ndFlrSF',
    'BsmtFinSF2',
    'SaleCondition',
    'PavedDrive',
    'LandContour',
    'Heating',
    'KitchenAbvGr',
    'BedroomAbvGr',
    'MSZoning',
    'Electrical',
    'TotalBsmtSF',
    'Fireplaces',
    'RoofStyle',
    'MasVnrType',
    'MasVnrArea',
    'BsmtUnfSF',
    'GarageCars',
    'Neighborhood',
    'WoodDeckSF',
    'Utilities',
    'CentralAir',
    'Condition1',
    'FullBath',
    'Foundation',
    'HalfBath',
    'ScreenPorch',
    'OpenPorchSF',
    'OverallQual',
    'Alley',
    'EnclosedPorch',
    'HouseStyle',
    'LowQualFinSF',
    'PoolArea',
    'ExterQual',
    'HeatingQC',
    'GarageQual',
    'GarageFinish',
    'BsmtHalfBath',
    'BsmtFinType2',
    'MSSubClass',
    '3SsnPorch',
    'BsmtFullBath',
    'MoSold',
    'PoolQC',
    'LotShape'
]
len(inclusion_cols_ordered)

73

In [ ]:
cols_included = []
test_scores = []
train_scores = []
for i in range(1,73):
  #Take Subset of training data columns
  cols_to_use = inclusion_cols_ordered[0:i]
  cols_to_use.append('SalePrice')
  sub_train = raw_data_train[cols_to_use]
  #Convert categorical categories to binary columns
  new_sub_train = convert_categorical_to_binary(sub_train, categorical_cols)
  #Loop over ordinal categories and convert them to numerical
  for col in ordinal_cols:
    new_sub_train = replace_categorical_with_numerical(new_sub_train, col, ordinal_cols[col])
  #Select Cols
  df = new_sub_train.select_dtypes(include = ['float64', 'int64']).fillna(0)
  y = df['SalePrice'].values
  X = df.drop('SalePrice',axis=1).values
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
  #Fit Data
  reg = LinearRegression().fit(X_train, y_train)
  test_score = reg.score(X_test,y_test)
  cols_included.append(i)
  test_scores.append(test_score)
  train_scores.append(reg.score(X_train,y_train))

In [ ]:
import plotly.express as px
r_squared_scores = pd.DataFrame(
    {'Features Used':cols_included, 'Test Data Scores':test_scores,'Training Data Scores':train_scores}
    )
stacked_df = r_squared_scores.melt('Features Used', value_name='R Squared Score', var_name='Data')
px.scatter(stacked_df, x='Features Used',y='R Squared Score',color='Data', title='R-squared score by number of features')